# MIS 381N: Information Management Final Project
## Team JAS - Google Trends Analysis
### Database: DB_Team_JAS | Role: ROLE_Team_JAS | Warehouse: Animal_Task_WH

---
## INITIAL SETUP
Set context and create schemas

In [ ]:
-- Set context
USE ROLE ROLE_Team_JAS;
USE DATABASE DB_Team_JAS;
USE WAREHOUSE Animal_Task_WH;

-- Create the three schemas
CREATE SCHEMA IF NOT EXISTS Bronze;
CREATE SCHEMA IF NOT EXISTS Silver;
CREATE SCHEMA IF NOT EXISTS Gold;

---
## TASK 1-2: BRONZE LAYER - RAW DATA IMPORT
Loading raw data without modifications

In [ ]:
USE SCHEMA Bronze;

-- List files in stage
LIST @bronze_stage;

In [ ]:
-- Create bronze table (raw data)
CREATE OR REPLACE TABLE google_trends_raw(
    query VARCHAR(500),
    start_date TIMESTAMP_NTZ,
    end_date TIMESTAMP_NTZ,
    active BOOLEAN,
    search_volume NUMBER(12,0),
    increase_percentage FLOAT,
    categories VARCHAR(1000),
    trend_breakdown VARCHAR(5000)
);

In [ ]:
-- Load data from the stage into the table
COPY INTO google_trends_raw (
    query,
    start_date,
    end_date,
    active,
    search_volume,
    increase_percentage,
    categories,
    trend_breakdown
)
FROM (
    SELECT 
        $1::VARCHAR,
        $2::TIMESTAMP_NTZ,
        $3::TIMESTAMP_NTZ,
        $4::BOOLEAN,
        $5::NUMBER,
        $6::FLOAT,
        $7::VARCHAR,
        $8::VARCHAR
    FROM @bronze_stage
)
ON_ERROR = 'CONTINUE'
PURGE = FALSE;

In [ ]:
-- Verify data upload
-- Check row count
SELECT COUNT(*) AS total_rows 
FROM google_trends_raw;

-- View first 10 rows
SELECT * 
FROM google_trends_raw 
LIMIT 10;

-- View sample of data by category
SELECT 
    categories,
    COUNT(*) AS query_count,
    AVG(search_volume) AS avg_volume,
    MAX(increase_percentage) AS max_increase
FROM google_trends_raw
GROUP BY categories
ORDER BY query_count DESC
LIMIT 10;

---
## TASK 3-5: SILVER LAYER - DIMENSIONAL MODEL
Creating dimension, fact, and bridge tables with star schema design

In [ ]:
USE SCHEMA Silver;

-- Drop existing tables if recreating
DROP TABLE IF EXISTS bridge_query_category;
DROP TABLE IF EXISTS fact_search_trends;

### Create Dimension Tables

In [ ]:
-- Dimension: Queries
CREATE OR REPLACE TABLE dim_query (
    query_key NUMBER AUTOINCREMENT PRIMARY KEY,
    query VARCHAR(500) NOT NULL,
    query_length NUMBER,
    word_count NUMBER,
    UNIQUE (query)
);

In [ ]:
-- Dimension: Categories
CREATE OR REPLACE TABLE dim_category (
    category_key NUMBER AUTOINCREMENT PRIMARY KEY,
    category_name VARCHAR(500) NOT NULL,
    UNIQUE (category_name)
);

In [ ]:
-- Dimension: Date
CREATE OR REPLACE TABLE dim_date (
    date_key NUMBER PRIMARY KEY,
    full_date DATE NOT NULL,
    year NUMBER,
    quarter NUMBER,
    month NUMBER,
    month_name VARCHAR(20),
    week NUMBER,
    day_of_month NUMBER,
    day_of_week NUMBER,
    day_name VARCHAR(20),
    is_weekend BOOLEAN,
    UNIQUE (full_date)
);

### Create Fact Table

In [ ]:
-- Fact Table: Search Trends
CREATE OR REPLACE TABLE fact_search_trends (
    trend_key NUMBER AUTOINCREMENT PRIMARY KEY,
    query_key NUMBER NOT NULL,
    category_key NUMBER,
    start_date_key NUMBER NOT NULL,
    end_date_key NUMBER,
    search_volume NUMBER(12,0),
    increase_percentage FLOAT,
    active BOOLEAN,
    trend_duration_days NUMBER,
    trend_breakdown VARCHAR(5000),
    load_timestamp TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
    source_system VARCHAR(50) DEFAULT 'Google Trends',
    FOREIGN KEY (query_key) REFERENCES dim_query(query_key),
    FOREIGN KEY (category_key) REFERENCES dim_category(category_key),
    FOREIGN KEY (start_date_key) REFERENCES dim_date(date_key)
);

### Create Bridge Table

In [ ]:
-- Bridge Table: Query-Category (handles many-to-many relationship)
CREATE OR REPLACE TABLE bridge_query_category (
    query_key NUMBER NOT NULL,
    category_key NUMBER NOT NULL,
    PRIMARY KEY (query_key, category_key),
    FOREIGN KEY (query_key) REFERENCES dim_query(query_key),
    FOREIGN KEY (category_key) REFERENCES dim_category(category_key)
);

### Populate Silver Layer from Bronze

In [ ]:
-- Populate dim_date
INSERT INTO dim_date (
    date_key,
    full_date,
    year,
    quarter,
    month,
    month_name,
    week,
    day_of_month,
    day_of_week,
    day_name,
    is_weekend
)
WITH date_range AS (
    SELECT DISTINCT DATE(start_date) AS date_value
    FROM Bronze.google_trends_raw
    WHERE start_date IS NOT NULL
    UNION
    SELECT DISTINCT DATE(end_date) AS date_value
    FROM Bronze.google_trends_raw
    WHERE end_date IS NOT NULL
)
SELECT 
    TO_NUMBER(TO_CHAR(date_value, 'YYYYMMDD')) AS date_key,
    date_value AS full_date,
    YEAR(date_value) AS year,
    QUARTER(date_value) AS quarter,
    MONTH(date_value) AS month,
    MONTHNAME(date_value) AS month_name,
    WEEK(date_value) AS week,
    DAY(date_value) AS day_of_month,
    DAYOFWEEK(date_value) AS day_of_week,
    DAYNAME(date_value) AS day_name,
    CASE WHEN DAYOFWEEK(date_value) IN (0, 6) THEN TRUE ELSE FALSE END AS is_weekend
FROM date_range;

In [ ]:
-- Populate dim_query
INSERT INTO dim_query (
    query,
    query_length,
    word_count
)
SELECT DISTINCT
    query,
    LENGTH(query) AS query_length,
    LENGTH(query) - LENGTH(REPLACE(query, ' ', '')) + 1 AS word_count
FROM Bronze.google_trends_raw
WHERE query IS NOT NULL;

In [ ]:
-- Populate dim_category
INSERT INTO dim_category (category_name)
WITH split_categories AS (
    SELECT DISTINCT
        TRIM(VALUE::STRING) AS category_name
    FROM Bronze.google_trends_raw,
    LATERAL FLATTEN(INPUT => SPLIT(categories, ','))
    WHERE categories IS NOT NULL
)
SELECT category_name
FROM split_categories
WHERE category_name != ''
ORDER BY category_name;

In [ ]:
-- Populate fact_search_trends
INSERT INTO fact_search_trends (
    query_key,
    category_key,
    start_date_key,
    end_date_key,
    search_volume,
    increase_percentage,
    active,
    trend_duration_days,
    trend_breakdown
)
SELECT 
    dq.query_key,
    dc.category_key,
    TO_NUMBER(TO_CHAR(DATE(b.start_date), 'YYYYMMDD')) AS start_date_key,
    CASE
        WHEN b.end_date IS NOT NULL
        THEN TO_NUMBER(TO_CHAR(DATE(b.end_date), 'YYYYMMDD')) 
        ELSE NULL
    END AS end_date_key,
    b.search_volume,
    b.increase_percentage,
    b.active,
    CASE 
        WHEN b.end_date IS NOT NULL
        THEN DATEDIFF(day, b.start_date, b.end_date) 
        ELSE NULL
    END AS trend_duration_days,
    b.trend_breakdown
FROM Bronze.google_trends_raw b
LEFT JOIN dim_query dq ON b.query = dq.query
LEFT JOIN dim_category dc ON TRIM(SPLIT_PART(b.categories, ',', 1)) = dc.category_name
WHERE b.query IS NOT NULL
    AND b.start_date IS NOT NULL;

In [ ]:
-- Populate bridge_query_category
INSERT INTO bridge_query_category (query_key, category_key)
WITH category_splits AS (
    SELECT 
        dq.query_key,
        TRIM(VALUE::STRING) AS category_name
    FROM Bronze.google_trends_raw b
    JOIN dim_query dq ON b.query = dq.query,
    LATERAL FLATTEN(INPUT => SPLIT(b.categories, ','))
    WHERE b.categories IS NOT NULL
)
SELECT DISTINCT
    cs.query_key,
    dc.category_key
FROM category_splits cs
JOIN dim_category dc ON cs.category_name = dc.category_name;

### Verify Silver Layer

In [ ]:
-- Check row counts for all Silver tables
SELECT 'dim_query' AS table_name, COUNT(*) AS row_count FROM dim_query
UNION ALL
SELECT 'dim_category', COUNT(*) FROM dim_category
UNION ALL
SELECT 'dim_date', COUNT(*) FROM dim_date
UNION ALL
SELECT 'fact_search_trends', COUNT(*) FROM fact_search_trends
UNION ALL
SELECT 'bridge_query_category', COUNT(*) FROM bridge_query_category;

In [ ]:
-- Sample fact table with dimensions joined (Star Schema)
SELECT 
    dq.query,
    dc.category_name,
    dd_start.full_date AS start_date,
    dd_start.month_name,
    CASE 
        WHEN f.end_date_key IS NOT NULL 
        THEN dd_end.full_date 
        ELSE NULL 
    END AS end_date,
    f.search_volume,
    f.increase_percentage,
    f.active,
    f.trend_duration_days
FROM fact_search_trends f
JOIN dim_query dq ON f.query_key = dq.query_key
LEFT JOIN dim_category dc ON f.category_key = dc.category_key
JOIN dim_date dd_start ON f.start_date_key = dd_start.date_key
LEFT JOIN dim_date dd_end ON f.end_date_key = dd_end.date_key
LIMIT 10;

---
## TASK 6-7: GOLD LAYER - BUSINESS REPORTING
Creating aggregate tables for 3 business use cases

In [ ]:
USE SCHEMA Gold;

### Use Case 1: Category Popularity Tracker
Aggregates total volume by category using bridge table

In [ ]:
CREATE OR REPLACE TABLE gold_category_metrics AS
SELECT 
    dc.category_name,
    COUNT(DISTINCT f.query_key) AS number_of_unique_trends,
    SUM(f.search_volume) AS total_search_volume,
    AVG(f.increase_percentage) AS avg_growth_pct
FROM Silver.fact_search_trends f
JOIN Silver.bridge_query_category bqc ON f.query_key = bqc.query_key
JOIN Silver.dim_category dc ON bqc.category_key = dc.category_key
GROUP BY dc.category_name
ORDER BY total_search_volume DESC;

### Use Case 2: Daily Search Volume Trends
Time-series data for line graphs

In [ ]:
CREATE OR REPLACE TABLE gold_daily_traffic AS
SELECT 
    dd.full_date AS report_date,
    dd.day_name,
    SUM(f.search_volume) AS total_daily_volume,
    AVG(f.increase_percentage) AS daily_avg_growth,
    COUNT(DISTINCT f.query_key) AS active_trends_count
FROM Silver.fact_search_trends f
JOIN Silver.dim_date dd ON f.start_date_key = dd.date_key
GROUP BY dd.full_date, dd.day_name
ORDER BY dd.full_date DESC;

### Use Case 3: Category Leaders (Top 10 per Category)
Ranks top queries within each category by volume

In [ ]:
CREATE OR REPLACE TABLE gold_category_leaders AS
SELECT * FROM (
    SELECT 
        dc.category_name,
        dq.query AS query_name,
        f.search_volume,
        f.increase_percentage,
        ROW_NUMBER() OVER (PARTITION BY dc.category_name ORDER BY f.search_volume DESC) as rank_in_category
    FROM Silver.fact_search_trends f
    JOIN Silver.dim_query dq ON f.query_key = dq.query_key
    JOIN Silver.dim_category dc ON f.category_key = dc.category_key
)
WHERE rank_in_category <= 10;

### Verify Gold Layer

In [ ]:
-- Preview Gold tables
SELECT * FROM gold_category_metrics LIMIT 5;
SELECT * FROM gold_daily_traffic LIMIT 5;
SELECT * FROM gold_category_leaders LIMIT 5;

---
## TASK 10: STREAMLIT DASHBOARD
3 visualizations from Gold Layer

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import altair as alt
from snowflake.snowpark.context import get_active_session

# Set Page Config
st.set_page_config(layout="wide")

# Get the current credentials
session = get_active_session()

# ==========================================
# HELPER FUNCTION TO LOAD DATA
# ==========================================
def load_data(table_name):
    query = f"SELECT * FROM Gold.{table_name}"
    return session.sql(query).to_pandas()

# ==========================================
# APP HEADER
# ==========================================
st.title("📊 Google Search Trends Dashboard")
st.write(
    """
    **Team JAS Analysis**
    This dashboard visualizes key search trends from our Gold Layer data, 
    focusing on Category Popularity, Daily Traffic, and Viral Breakouts.
    """
)
st.divider()

# Create Tabs for better organization
tab1, tab2, tab3 = st.tabs(["🏆 Category Popularity", "📈 Daily Trends", "🔥 Viral Breakouts"])

# ==========================================
# TAB 1: CATEGORY POPULARITY (Bar Chart)
# ==========================================
with tab1:
    st.header("Which topics are driving the most traffic?")
    
    # Load Data
    df_category = load_data("gold_category_metrics")
    
    # Display Chart
    chart = alt.Chart(df_category).mark_bar().encode(
        x=alt.X('TOTAL_SEARCH_VOLUME', title='Total Search Volume'),
        y=alt.Y('CATEGORY_NAME', sort='-x', title='Category'),
        color=alt.Color('CATEGORY_NAME', legend=None)
    ).properties(height=400)
    
    st.altair_chart(chart, use_container_width=True)
    
    # Optional: Show raw data toggle
    with st.expander("View Source Data"):
        st.dataframe(df_category, use_container_width=True)

# ==========================================
# TAB 2: DAILY TRAFFIC (Line Chart)
# ==========================================
with tab2:
    st.header("How does search volume change over time?")
    
    # Load Data
    df_daily = load_data("gold_daily_traffic")
    
    # Ensure date column is datetime format
    df_daily['REPORT_DATE'] = pd.to_datetime(df_daily['REPORT_DATE'])
    
    # Simple Streamlit Line Chart
    st.line_chart(
        data=df_daily,
        x='REPORT_DATE',
        y='TOTAL_DAILY_VOLUME',
        color='#FF4B4B'
    )
    
    # Metric Summary
    total_vol = df_daily['TOTAL_DAILY_VOLUME'].sum()
    peak_day = df_daily.loc[df_daily['TOTAL_DAILY_VOLUME'].idxmax()]
    
    col1, col2 = st.columns(2)
    col1.metric("Total Volume Recorded", f"{total_vol:,}")
    col2.metric("Peak Traffic Day", f"{peak_day['REPORT_DATE'].strftime('%Y-%m-%d')}")

# ==========================================
# TAB 3: CATEGORY DEEP DIVE (Drill Down)
# ==========================================
with tab3:
    st.header("🔎 Category Deep Dive")
    st.caption("Select a category below to see exactly what is driving the traffic.")

    # Load the new Gold Table
    df_leaders = load_data("gold_category_leaders")

    # CREATE A DROPDOWN MENU
    category_list = df_leaders['CATEGORY_NAME'].unique()
    selected_category = st.selectbox("Choose a Topic:", category_list)

    # Filter data based on selection
    filtered_df = df_leaders[df_leaders['CATEGORY_NAME'] == selected_category]
    filtered_df = filtered_df.sort_values(by='SEARCH_VOLUME', ascending=False)

    # Display the Bar Chart
    chart = alt.Chart(filtered_df).mark_bar().encode(
        x=alt.X('SEARCH_VOLUME', title='Search Volume'),
        y=alt.Y('QUERY_NAME', sort='-x', title='Search Term'),
        color=alt.value('#29b5e8'),
        tooltip=['QUERY_NAME', 'SEARCH_VOLUME', 'INCREASE_PERCENTAGE']
    ).properties(
        height=400,
        title=f"Top 10 Trends in {selected_category}"
    )

    st.altair_chart(chart, use_container_width=True)

    # Insight metric
    top_driver = filtered_df.iloc[0]
    st.success(f"💡 The #1 driver for **{selected_category}** is **'{top_driver['QUERY_NAME']}'** with {top_driver['SEARCH_VOLUME']:,} searches.")

---
## TASK 8-10: INCREMENTAL DATA LOAD
Loading new data into Bronze → Silver → Gold layers

In [ ]:
USE ROLE ROLE_Team_JAS;
USE DATABASE DB_Team_JAS;
USE WAREHOUSE Animal_Task_WH;

### Bronze Layer - Incremental Load

In [ ]:
USE SCHEMA Bronze;

-- Check BRONZE row count BEFORE loading new file(s)
SELECT COUNT(*) AS bronze_rows_before
FROM GOOGLE_TRENDS_RAW;

In [ ]:
-- Load new file(s) from the Bronze stage
COPY INTO GOOGLE_TRENDS_RAW (
    QUERY,
    START_DATE,
    END_DATE,
    ACTIVE,
    SEARCH_VOLUME,
    INCREASE_PERCENTAGE,
    CATEGORIES,
    TREND_BREAKDOWN
)
FROM (
    SELECT 
        $1::VARCHAR,
        $2::TIMESTAMP_NTZ,
        $3::TIMESTAMP_NTZ,
        $4::BOOLEAN,
        $5::NUMBER,
        $6::FLOAT,
        $7::VARCHAR,
        $8::VARCHAR
    FROM @BRONZE_STAGE
)
ON_ERROR = 'CONTINUE';

In [ ]:
-- Check BRONZE row count AFTER loading
SELECT COUNT(*) AS bronze_rows_after
FROM GOOGLE_TRENDS_RAW;

### Silver Layer - Incremental Load

In [ ]:
USE SCHEMA Silver;

-- Incremental load into DIM_DATE
INSERT INTO DIM_DATE (
    DATE_KEY,
    FULL_DATE,
    YEAR,
    QUARTER,
    MONTH,
    MONTH_NAME,
    WEEK,
    DAY_OF_MONTH,
    DAY_OF_WEEK,
    DAY_NAME,
    IS_WEEKEND
)
WITH new_date_range AS (
    SELECT DISTINCT DATE(b.START_DATE) AS date_value
    FROM DB_Team_JAS.BRONZE.GOOGLE_TRENDS_RAW b
    LEFT JOIN DIM_DATE d ON d.FULL_DATE = DATE(b.START_DATE)
    WHERE b.START_DATE IS NOT NULL AND d.FULL_DATE IS NULL
    UNION
    SELECT DISTINCT DATE(b.END_DATE) AS date_value
    FROM DB_Team_JAS.BRONZE.GOOGLE_TRENDS_RAW b
    LEFT JOIN DIM_DATE d ON d.FULL_DATE = DATE(b.END_DATE)
    WHERE b.END_DATE IS NOT NULL AND d.FULL_DATE IS NULL
)
SELECT 
    TO_NUMBER(TO_CHAR(date_value, 'YYYYMMDD')) AS DATE_KEY,
    date_value AS FULL_DATE,
    YEAR(date_value) AS YEAR,
    QUARTER(date_value) AS QUARTER,
    MONTH(date_value) AS MONTH,
    MONTHNAME(date_value) AS MONTH_NAME,
    WEEK(date_value) AS WEEK,
    DAY(date_value) AS DAY_OF_MONTH,
    DAYOFWEEK(date_value) AS DAY_OF_WEEK,
    DAYNAME(date_value) AS DAY_NAME,
    CASE WHEN DAYOFWEEK(date_value) IN (0,6) THEN TRUE ELSE FALSE END AS IS_WEEKEND
FROM new_date_range;

In [ ]:
-- Incremental load into DIM_QUERY
INSERT INTO DIM_QUERY (QUERY, QUERY_LENGTH, WORD_COUNT)
SELECT DISTINCT
    b.QUERY,
    LENGTH(b.QUERY),
    LENGTH(b.QUERY) - LENGTH(REPLACE(b.QUERY, ' ', '')) + 1
FROM DB_Team_JAS.BRONZE.GOOGLE_TRENDS_RAW b
LEFT JOIN DIM_QUERY dq ON dq.QUERY = b.QUERY
WHERE b.QUERY IS NOT NULL AND dq.QUERY IS NULL;

In [ ]:
-- Incremental load into DIM_CATEGORY
INSERT INTO DIM_CATEGORY (CATEGORY_NAME)
WITH exploded_categories AS (
    SELECT DISTINCT
        TRIM(f.VALUE::STRING) AS category_name
    FROM DB_Team_JAS.BRONZE.GOOGLE_TRENDS_RAW b,
         LATERAL FLATTEN(INPUT => SPLIT(b.CATEGORIES, ',')) f
)
SELECT e.category_name
FROM exploded_categories e
LEFT JOIN DIM_CATEGORY dc ON dc.CATEGORY_NAME = e.category_name
WHERE e.category_name IS NOT NULL
  AND e.category_name != ''
  AND dc.CATEGORY_NAME IS NULL;

In [ ]:
-- Incremental load into BRIDGE_QUERY_CATEGORY
INSERT INTO BRIDGE_QUERY_CATEGORY (QUERY_KEY, CATEGORY_KEY)
WITH category_splits AS (
    SELECT 
        dq.QUERY_KEY,
        TRIM(f.VALUE::STRING) AS category_name
    FROM DB_Team_JAS.BRONZE.GOOGLE_TRENDS_RAW b
    JOIN DIM_QUERY dq ON b.QUERY = dq.QUERY,
    LATERAL FLATTEN(INPUT => SPLIT(b.CATEGORIES, ',')) f
    WHERE b.CATEGORIES IS NOT NULL
),
candidate_pairs AS (
    SELECT DISTINCT
        cs.QUERY_KEY,
        dc.CATEGORY_KEY
    FROM category_splits cs
    JOIN DIM_CATEGORY dc ON cs.category_name = dc.CATEGORY_NAME
)
SELECT cp.QUERY_KEY, cp.CATEGORY_KEY
FROM candidate_pairs cp
LEFT JOIN BRIDGE_QUERY_CATEGORY bqc 
  ON bqc.QUERY_KEY = cp.QUERY_KEY AND bqc.CATEGORY_KEY = cp.CATEGORY_KEY
WHERE bqc.QUERY_KEY IS NULL;

In [ ]:
-- FACT_SEARCH_TRENDS row count BEFORE incremental load
SELECT COUNT(*) AS fact_rows_before
FROM FACT_SEARCH_TRENDS;

In [ ]:
-- Incremental load into FACT_SEARCH_TRENDS
INSERT INTO FACT_SEARCH_TRENDS (
    QUERY_KEY,
    CATEGORY_KEY,
    START_DATE_KEY,
    END_DATE_KEY,
    SEARCH_VOLUME,
    INCREASE_PERCENTAGE,
    ACTIVE,
    TREND_DURATION_DAYS,
    TREND_BREAKDOWN
)
SELECT 
    dq.QUERY_KEY,
    dc.CATEGORY_KEY,
    TO_NUMBER(TO_CHAR(DATE(b.START_DATE), 'YYYYMMDD')) AS START_DATE_KEY,
    CASE
        WHEN b.END_DATE IS NOT NULL
        THEN TO_NUMBER(TO_CHAR(DATE(b.END_DATE), 'YYYYMMDD'))
        ELSE NULL
    END AS END_DATE_KEY,
    b.SEARCH_VOLUME,
    b.INCREASE_PERCENTAGE,
    b.ACTIVE,
    CASE 
        WHEN b.END_DATE IS NOT NULL
        THEN DATEDIFF('day', b.START_DATE, b.END_DATE)
        ELSE NULL
    END AS TREND_DURATION_DAYS,
    b.TREND_BREAKDOWN
FROM DB_Team_JAS.BRONZE.GOOGLE_TRENDS_RAW b
LEFT JOIN DIM_QUERY dq ON b.QUERY = dq.QUERY
LEFT JOIN DIM_CATEGORY dc ON TRIM(SPLIT_PART(b.CATEGORIES, ',', 1)) = dc.CATEGORY_NAME
LEFT JOIN FACT_SEARCH_TRENDS f_existing
  ON f_existing.QUERY_KEY = dq.QUERY_KEY
 AND f_existing.START_DATE_KEY = TO_NUMBER(TO_CHAR(DATE(b.START_DATE), 'YYYYMMDD'))
 AND NVL(f_existing.END_DATE_KEY, -1) = NVL(
        CASE
            WHEN b.END_DATE IS NOT NULL
            THEN TO_NUMBER(TO_CHAR(DATE(b.END_DATE), 'YYYYMMDD'))
            ELSE NULL
        END, -1)
WHERE b.QUERY IS NOT NULL
  AND b.START_DATE IS NOT NULL
  AND f_existing.TREND_KEY IS NULL;

In [ ]:
-- FACT_SEARCH_TRENDS row count AFTER incremental load
SELECT COUNT(*) AS fact_rows_after
FROM FACT_SEARCH_TRENDS;

### Gold Layer - Refresh Aggregate Tables

In [ ]:
USE SCHEMA Gold;

-- Rebuild GOLD_CATEGORY_METRICS
CREATE OR REPLACE TABLE GOLD_CATEGORY_METRICS AS
SELECT 
    dc.CATEGORY_NAME,
    COUNT(DISTINCT f.QUERY_KEY) AS number_of_unique_trends,
    SUM(f.SEARCH_VOLUME) AS total_search_volume,
    AVG(f.INCREASE_PERCENTAGE) AS avg_growth_pct
FROM SILVER.FACT_SEARCH_TRENDS f
JOIN SILVER.BRIDGE_QUERY_CATEGORY bqc ON f.QUERY_KEY = bqc.QUERY_KEY
JOIN SILVER.DIM_CATEGORY dc ON bqc.CATEGORY_KEY = dc.CATEGORY_KEY
GROUP BY dc.CATEGORY_NAME;

In [ ]:
-- Rebuild GOLD_DAILY_TRAFFIC
CREATE OR REPLACE TABLE GOLD_DAILY_TRAFFIC AS
SELECT 
    dd.FULL_DATE AS report_date,
    dd.DAY_NAME,
    SUM(f.SEARCH_VOLUME) AS total_daily_volume,
    AVG(f.INCREASE_PERCENTAGE) AS daily_avg_growth,
    COUNT(DISTINCT f.QUERY_KEY) AS active_trends_count
FROM SILVER.FACT_SEARCH_TRENDS f
JOIN SILVER.DIM_DATE dd ON f.START_DATE_KEY = dd.DATE_KEY
GROUP BY dd.FULL_DATE, dd.DAY_NAME;

In [ ]:
-- Rebuild GOLD_CATEGORY_LEADERS
CREATE OR REPLACE TABLE GOLD_CATEGORY_LEADERS AS
SELECT * 
FROM (
    SELECT 
        dc.CATEGORY_NAME,
        dq.QUERY AS query_name,
        f.SEARCH_VOLUME,
        f.INCREASE_PERCENTAGE,
        ROW_NUMBER() OVER (
            PARTITION BY dc.CATEGORY_NAME 
            ORDER BY f.SEARCH_VOLUME DESC
        ) AS rank_in_category
    FROM SILVER.FACT_SEARCH_TRENDS f
    JOIN SILVER.DIM_QUERY dq ON f.QUERY_KEY = dq.QUERY_KEY
    JOIN SILVER.DIM_CATEGORY dc ON f.CATEGORY_KEY = dc.CATEGORY_KEY
)
WHERE rank_in_category <= 10;

---
## TASK 11: AI/ML FUNCTIONS ON BRONZE LAYER
Using Snowflake Cortex functions for sentiment analysis and summarization

In [ ]:
USE ROLE ROLE_Team_JAS;
USE DATABASE DB_Team_JAS;
USE SCHEMA Bronze;
USE WAREHOUSE Animal_Task_WH;

In [ ]:
-- Check original Bronze table structure
DESCRIBE TABLE GOOGLE_TRENDS_RAW;

In [ ]:
-- Create AI-enriched Bronze table with sentiment and summary columns
CREATE OR REPLACE TABLE GOOGLE_TRENDS_RAW AS
SELECT
    -- Original columns
    QUERY,
    START_DATE,
    END_DATE,
    ACTIVE,
    SEARCH_VOLUME,
    INCREASE_PERCENTAGE,
    CATEGORIES,
    TREND_BREAKDOWN,
    -- AI/ML columns using Cortex
    SNOWFLAKE.CORTEX.SENTIMENT(TREND_BREAKDOWN) AS TREND_SENTIMENT_SCORE,
    SNOWFLAKE.CORTEX.SUMMARIZE(TREND_BREAKDOWN) AS TREND_SUMMARY
FROM GOOGLE_TRENDS_RAW;

In [ ]:
-- Inspect structure of the new table
DESCRIBE TABLE GOOGLE_TRENDS_RAW_AI;

-- Sample rows showing AI-generated columns
SELECT 
    QUERY,
    TREND_BREAKDOWN,
    TREND_SENTIMENT_SCORE,
    TREND_SUMMARY
FROM GOOGLE_TRENDS_RAW_AI
LIMIT 10;

---
## TASK 12: CORTEX SEARCH SERVICE
Create search service on Bronze layer descriptive column

In [ ]:
USE ROLE ROLE_Team_JAS;
USE DATABASE DB_Team_JAS;
USE SCHEMA BRONZE;
USE WAREHOUSE Animal_Task_WH;

SHOW CORTEX SEARCH SERVICES;

-- AI trends
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'DB_Team_JAS.BRONZE.GOOGLE_TRENDS_CSS',
    '{
      "query": "artificial intelligence",
      "columns": ["QUERY", "CATEGORIES", "TREND_BREAKDOWN", "TREND_SENTIMENT_SCORE"],
      "limit": 5
    }'
  )
)['results'] AS results;

--Sports
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'DB_Team_JAS.BRONZE.GOOGLE_TRENDS_CSS',
    '{
      "query": "sports",
      "columns": ["QUERY", "CATEGORIES", "TREND_BREAKDOWN", "TREND_SUMMARY"],
      "limit": 5
    }'
  )
)['results'] AS results;

-- Holiday/Shopping
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'DB_Team_JAS.BRONZE.GOOGLE_TRENDS_CSS',
    '{
      "query": "holiday shopping",
      "columns": ["QUERY", "CATEGORIES", "TREND_BREAKDOWN", "TREND_SENTIMENT_SCORE", "TREND_SUMMARY"],
      "limit": 5
    }'
  )
)['results'] AS results;

SHOW VIEWS IN SCHEMA SILVER;


---
## TASK 13: CORTEX ANALYST SERVICE
Natural language queries on Silver layer structured data

In [ ]:
-- Note: Cortex Analyst service creation and queries are typically done
-- through Snowflake UI or API
-- This cell documents the configuration and test questions used:
--
-- Service Name: google_trends_analyst
-- Target Tables: Silver layer (fact_search_trends + dimensions)
--
-- Example business questions asked:
-- 1. "What are the top 5 categories by search volume?"
-- 2. "Show me trends with highest growth percentage"
-- 3. "What is the average search volume by day of week?"

---
## PROJECT SUMMARY

This notebook contains all code for the Team JAS Google Trends Analysis project:

**Bronze Layer**: Raw data ingestion (16,557 records)

**Silver Layer**: Star schema with 4 dimensions, 1 fact table, 1 bridge table

**Gold Layer**: 3 business reporting tables
- Category popularity metrics
- Daily traffic trends  
- Top 10 queries per category

**Streamlit Dashboard**: 3 interactive visualizations

**Incremental Load**: Automated pipeline for new data

**AI/ML Functions**: Sentiment analysis and summarization on Bronze layer

**Cortex Services**: Search and Analyst tools configured

**Dataset Timeframe**: September 18 - October 21, 2025 (33 days)

**Team**: JAS | **Database**: DB_Team_JAS | **Warehouse**: Animal_Task_WH